# タスク 2a: Titan Text Premier を使用した小さなファイルによるテキスト要約

このノートブックでは、小さなテキスト文字列を Amazon Bedrock API (Titan Text モデルを使用) に直接取り込み、入力テキストを要約するように指示します。入力テキストの長さがモデルのコンテキストサイズ制限内であれば、このアプローチを適用して、通話記録、会議記録、書籍、記事、ブログ投稿、その他の関連コンテンツを要約できます。


## タスク 2a.1: 環境のセットアップ

このタスクでは、環境をセットアップします。

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys
import warnings

import boto3
import botocore

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))



## タスク 2a.2: 要約するテキストを含むプロンプトの作成

このタスクでは、基盤モデルでサポートされている最大長よりも少ないトークンを含む短いテキストを使用します。このラボのサンプル入力テキストとして、Amazon Bedrock を発表する [AWS ブログ投稿](https://aws.amazon.com/jp/blogs/machine-learning/announcing-new-tools-for-building-with-generative-ai-on-aws/) の段落を使用します。

プロンプトは、 `Please provide a summary of the following text.` という指示で始まります。

In [ ]:
prompt_data = """

Please provide a summary of the following text:

AWS took all of that feedback from customers, and today we are excited to announce Amazon Bedrock, \
a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. \
Bedrock is the easiest way for customers to build and scale generative AI-based applications using FMs, \
democratizing access for all builders. Bedrock will offer the ability to access a range of powerful FMs \
for text and images—including Amazons Titan FMs, which consist of two new LLMs we’re also announcing \
today—through a scalable, reliable, and secure AWS managed service. With Bedrock’s serverless experience, \
customers can easily find the right model for what they’re trying to get done, get started quickly, privately \
customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS \
tools and capabilities they are familiar with, without having to manage any infrastructure (including integrations \
with Amazon SageMaker ML features like Experiments to test different models and Pipelines to manage their FMs at scale).
"""

## タスク 2a.3: プロンプトと推論パラメータを含むリクエスト本文の作成

このタスクでは、上記のプロンプトと推論パラメータを含むリクエスト本文を作成します。

In [ ]:
# request body
body = json.dumps({
    "inputText": prompt_data, 
    "textGenerationConfig":{
        "maxTokenCount":2048,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
        }
    }) 

## タスク 2a.4: Boto3 経由で基盤モデルを呼び出す

このタスクでは、リクエストパラメータ `modelId`、`accept`、および `contentType` を指定して、Amazon Bedrock に API リクエストを送信します。提供されたプロンプトに続いて、Amazon Bedrock の基盤モデルが入力テキストを要約します。

### 完全な出力生成

デフォルトでは、Amazon Bedrock サービスは、特定のプロンプトの要約全体を 1 つの出力で生成します。モデル出力に多くのトークンが含まれている場合、出力する速度が遅くなる可能性があります。

In [ ]:
#model configuration and invoke the model
modelId = 'amazon.titan-text-premier-v1:0' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
outputText = "\n"

try:

    response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    response_body = json.loads(response.get('body').read())

    outputText = response_body.get('results')[0].get('outputText')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error

print(outputText)

### ストリーミング出力生成

次に、Amazon Bedrock のinvoke_model_with_response_stream API を使用してモデル出力をストリーミングし、ユーザーが生成された出力を消費できるようにする方法を探索します。この API は、完全な出力を一度に生成するのではなく、モデルから生成される小さな出力チャンクを送信する ResponseStream を返します。これらのストリーミング出力は、連続した消費可能なビューで表示できます。

In [ ]:
#invoke model with response stream
modelId = 'amazon.titan-text-premier-v1:0'
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = list(stream)
output

In [ ]:
from IPython.display import display_markdown,Markdown,clear_output

In [ ]:
modelId = 'amazon.titan-text-premier-v1:0'
response = bedrock_client.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

これで、boto3 SDK を使用して Amazon Bedrock API にアクセスする実験が完了しました。この SDK は、Bedrock 機能への基本的なプログラムによるアクセスを提供します。この API を活用することで、1) AWS ニュースコンテンツのテキスト要約全体を一度に生成する、2) 要約出力をチャンクでストリーミングして増分処理するという 2 つのユースケースを実装できました。

### 試してみましょう
- 特定のユースケースに合わせてプロンプトを変更し、さまざまなモデルの出力を評価します。
- トークンの長さを変えることで、サービスのレイテンシと応答性がどのように変化するかを理解します。
- さまざまなプロンプトエンジニアリングの原則を適用して、より良い出力を取得します。

### クリーンアップ

あなたはこのノートブックを完了しました。ラボの次のパートに移るには、下記を実行してください。:

- このノートブックファイルを閉じ、 **Task2b_ja_jp.ipynb** を開きます。
